In [2]:
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09c_changesig.ipynb"
_nb_title = "내보낸 모델의 시그니처 바꾸기"
_nb_message = ""

### no need to change any of this
_icons=["https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/ychoi-kr/practical-ml-vision-book-ko/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Google Colab에서 실행</a></td><td><a target="_blank" href="{2}"><img src="{6}" />GitHub에서 소스 보기</a></td><td><a href="{3}"><img src="{7}" />노트북 내려받기</a></td></table><br/><br/><h1>{8}</h1>{9}""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3], _nb_title, _nb_message))

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=%EB%82%B4%EB%B3%B4%EB%82%B8+%EB%AA%A8%EB%8D%B8%EC%9D%98+%EC%8B%9C%EA%B7%B8%EB%8B%88%EC%B2%98+%EB%B0%94%EA%BE%B8%EA%B8%B0&download_url=https%3A%2F%2Fgithub.com%2Fychoi-kr%2Fpractical-ml-vision-book-ko%2Fraw%2Fmaster%2F09_deploying%2F09c_changesig.ipynb"><img src="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Google Colab에서 실행</a></td><td><a target="_blank" href="https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub에서 소스 보기</a></td><td><a href="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/09_deploying/09c_changesig.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 내려받기</a></td></table><br/><br/><h1>내보낸 모델의 시그니처 바꾸기</h1>

# 내보낸 모델의 시그니처 바꾸기

이 노트북에서는 앞서 훈련 및 저장한 모델(7장)을 가지고 시작한다.
편의상 이 모델을 공개 버킷 [gs://practical-ml-vision-book/flowers_5_trained](gs://practical-ml-vision-book/flowers_5_trained)에 두었다.


## GPU 활성화 및 도우미 함수 셋업

이 저장소의 노트북들은 GPU를 사용하면 더 빨리 실행된다.

코랩에서:

- 수정 → 노트 설정 메뉴로 이동
- 하드웨어 가속기 드롭다운에서 GPU를 선택

Cloud AI Platform Notebooks에서:

- [https://console.cloud.google.com/ai-platform/notebooks](https://console.cloud.google.com/ai-platform/notebooks)로 이동
- GPU를 포함한 인스턴스를 생성하거나, 인스턴스를 선택하고 GPU를 추가

다음으로, 텐서플로로 GPU에 연결할 수 있는지 확인.

In [ ]:
import tensorflow as tf
print('TensorFlow version' + tf.version.VERSION)
print('Built with GPU support? ' + ('Yes!' if tf.test.is_built_with_cuda() else 'Noooo!'))
print('There are {} GPUs'.format(len(tf.config.experimental.list_physical_devices("GPU"))))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow version2.6.2
Built with GPU support? Yes!
There are 1 GPUs
Found GPU at: /device:GPU:0


2021-12-05 02:36:35.367935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:35.376968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:35.377557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:35.378801: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## 내보낸 모델

7장에서 훈련 및 저장한 모델을 가지고 시작한다.

<pre>
  model.save(...)
</pre>

In [ ]:
MODEL_LOCATION='gs://practical-ml-vision-book/flowers_5_trained'

In [ ]:
!gsutil ls {MODEL_LOCATION}

gs://practical-ml-vision-book/flowers_5_trained/saved_model.pb
gs://practical-ml-vision-book/flowers_5_trained/chkpts/
gs://practical-ml-vision-book/flowers_5_trained/variables/


In [ ]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir {MODEL_LOCATION}

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Method name is: tensorflow/serving/predict


## Passing through an input

시그니처에는 입력 파일 이름이 없다. 추가하자.

In [ ]:
import os
import shutil
import tensorflow as tf
model = tf.keras.models.load_model(MODEL_LOCATION)

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_flower_type(filenames):
    old_fn = model.signatures['serving_default']
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model',
          signatures={
              'serving_default': predict_flower_type
          })

2021-12-05 02:36:42.073546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:42.074258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:42.074816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:42.075456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-05 02:36:42.076125: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

INFO:tensorflow:Assets written to: export/flowers_model/assets


In [ ]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir export/flowers_model

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:3
Method name is: tensorflow/serving/predict


In [ ]:
import tensorflow as tf
serving_fn = tf.keras.models.load_model('export/flowers_model').signatures['serving_default']
filenames = [
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
pred = serving_fn(tf.convert_to_tensor(filenames))
print(pred)

2021-12-05 02:37:09.518516: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


{'flower_type_str': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'dandelion', b'dandelion', b'daisy', b'tulips', b'tulips'],
      dtype=object)>, 'filename': <tf.Tensor: shape=(5,), dtype=string, numpy=
array([b'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
       b'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
       b'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
       b'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
       b'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'],
      dtype=object)>, 'probability': <tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.6191508 , 0.99998426, 0.99508286, 0.97518593, 0.9549181 ],
      dtype=float32)>, 'flower_type_int': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 0, 4, 4])>}


## 다중 시그니처


In [ ]:
import os
import shutil
import tensorflow as tf
model = tf.keras.models.load_model(MODEL_LOCATION)
old_fn = model.signatures['serving_default']

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def pass_through_input(filenames):
    result = old_fn(filenames) # has flower_type_int etc.
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model2',
          signatures={
              'serving_default': old_fn,
              'input_pass_through': pass_through_input
          })

INFO:tensorflow:Assets written to: export/flowers_model2/assets


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "input_pass_through"
SignatureDef key: "serving_default"


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:2
Method name is: tensorflow/serving/predict


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model2 --signature_def input_pass_through

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: input_pass_through_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:3
Method name is: tensorflow/serving/predict


## 다중 시그니처 모델을 REST API로 배포

In [ ]:
%%bash
BUCKET="ai-analytics-solutions-mlvisionbook" # CHANGE
gsutil -m cp -r ./export/flowers_model2 gs://${BUCKET}/flowers_model2

Copying file://./export/flowers_model2/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model2/saved_model.pb [Content-Type=application/octet-stream]...
/ [4/4 files][ 10.9 MiB/ 10.9 MiB] 100% Done                                    
Operation completed over 4 objects/10.9 MiB.                                     


In [ ]:
%%bash
BUCKET="ai-analytics-solutions-mlvisionbook" # CHANGE
./vertex_deploy.sh \
--endpoint_name=multi \
--model_name=multi \
--model_location=gs://${BUCKET}/flowers_model2

Deploying model multi
Creating multi endpoint now.
The endpoint_id is 130472447798411264
Uploading multi model now.
The model_id is 8769849878676242432
Deploying model now


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [1944520467301793792]...
......done.
Created Vertex AI endpoint: projects/379218021631/locations/us-central1/endpoints/130472447798411264.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [6376062500634361856]...
......................done.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [2946571384391729152]...
.............................................................................................................................................................................................................................................................................

## 중요: 이 셀을 변경

위의 엔드포인트 ID와 배포된 모델 ID를 기록해 둔다. 아래 셀에 입력한다.

In [ ]:
# 배포한 모델에 맞게 아래를 수정할 것
import os
os.environ['ENDPOINT_ID'] = '130472447798411264' # 바
os.environ['MODEL_ID'] = '1810350293179695104' # 꿔
os.environ['PROJECT'] = 'ai-analytics-solutions' # 주
os.environ['BUCKET'] = 'ai-analytics-solutions-mlvisionbook' # 세
os.environ['REGION'] = 'us-central1' # 요

In [ ]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [ ]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "1810350293179695104",
  "model": "projects/379218021631/locations/us-central1/models/8769849878676242432",
  "modelDisplayName": "multi",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


In [ ]:
%%writefile request.json
{
    "signature_name": "input_pass_through",
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [ ]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "1810350293179695104",
  "model": "projects/379218021631/locations/us-central1/models/8769849878676242432",
  "modelDisplayName": "multi",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


이것은 버그다. 리포트했다. 빨리 고쳐지면 좋겠다.

In [ ]:
# 파이썬에서 호출
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "signature_name": "input_pass_through",  # currently bugged
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "error": {\n    "code": 400,\n    "message": "Invalid JSON payload received. Unknown name \\"signature_name\\": Cannot find field.",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.BadRequest",\n        "fieldViolations": [\n          {\n            "description": "Invalid JSON payload received. Unknown name \\"signature_name\\": Cannot find field."\n          }\n        ]\n      }\n    ]\n  }\n}\n'


## 라이선스
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.